![](https://i.imgur.com/eBRPvWB.png)

# Bahdanau Model

Implementation of RNN sequence-to-sequence model from Bahdanau et al. as a basis for more sophisticated models such as reinforced, hierarchical-attention, or topic-focused summarization.

**Note**: We use the CNN/Dailymail dataset from [@JafferWilson](https://github.com/JafferWilson/Process-Data-of-CNN-DailyMail)'s Github account. 

# The Sequence to Sequence model

A [Sequence to Sequence network](http://arxiv.org/abs/1409.3215), or seq2seq network, or [Encoder Decoder network](https://arxiv.org/pdf/1406.1078v3.pdf), is a model consisting of two separate RNNs called the **encoder** and **decoder**. The encoder reads an input sequence one item at a time, and outputs a vector at each step. The final output of the encoder is kept as the **context** vector. The decoder uses this context vector to produce a sequence of outputs one step at a time.

![](https://i.imgur.com/tVtHhNp.png)

## The Attention Mechanism

The fixed-length vector carries the burden of encoding the the entire "meaning" of the input sequence, no matter how long that may be. With all the variance in language, this is a very hard problem. Imagine two nearly identical sentences, twenty words long, with only one word different. Both the encoders and decoders must be nuanced enough to represent that change as a very slightly different point in space.

The **attention mechanism** [introduced by Bahdanau et al.](https://arxiv.org/abs/1409.0473) addresses this by giving the decoder a way to "pay attention" to parts of the input, rather than relying on a single vector. For every step the decoder can select a different part of the input sentence to consider.

![](https://i.imgur.com/5y6SCvU.png)

Attention is calculated with another feedforward layer in the decoder. This layer will use the current input and hidden state to create a new vector, which is the same size as the input sequence (in practice, a fixed maximum length). This vector is processed through softmax to create *attention weights*, which are multiplied by the encoders' outputs to create a new context vector, which is then used to predict the next output.

![](https://i.imgur.com/K1qMPxs.png)

# Requirements

You will need [PyTorch](http://pytorch.org/) to build and train the models, and [matplotlib](https://matplotlib.org/) for plotting training and visualizing attention outputs later.

In [1]:
import unicodedata
import string
import re
import random
import time
import math
import csv

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="6,7"


Here we will also define a constant to decide whether to use the GPU (with CUDA specifically) or the CPU. **If you don't have a GPU, set this to `False`**. Later when we create tensors, this variable will be used to decide whether we keep them on CPU or move them to GPU.

In [2]:
USE_CUDA = False

In [3]:
print(torch.__version__)

0.3.0


# Loading Data Files

Similar to the character encoding used in the character-level RNN tutorials, we will be representing each word in a language as a one-hot vector, or giant vector of zeros except for a single one (at the index of the word). Compared to the dozens of characters that might exist in a language, there are many many more words, so the encoding vector is much larger. We will however cheat a bit and trim the data to only use a few thousand words per language. On other tutorials,  they adopt an initial embedding matrix built from [GloVe](http://nlp.stanford.edu/projects/glove/), which can be tested afterwards.

### Indexing words

We'll need a unique index per word to use as the inputs and targets of the networks later. To keep track of all this we will use a helper class called `Voc` which has word &rarr; index (`word2index`) and index &rarr; word (`index2word`) dictionaries, as well as a count of each word `word2count` to use to later replace rare words.

In [4]:
SOS_token = 0
EOS_token = 1

class Voc:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2 # Count SOS and EOS
      
    def index_words(self, sentence):
        for word in sentence.split(' '):
            self.index_word(word)

    def index_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

### Reading and decoding files

The files are all in Unicode, to simplify we will turn Unicode characters to ASCII, make everything lowercase, and trim most punctuation.

In [5]:
# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalize_string(s):
    s = unicode_to_ascii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data file we will split the file into lines, and then split lines into pairs. The files are all description &rarr; headline, so if we want to generate text from headline &rarr; description I added the `reverse` flag to reverse the pairs.

In [6]:
def read_csv(csvname, reverse=False):
    print("Reading csv...")
    
    # Read the file and split into lines
    with open('../data/%s.csv' % csvname, 'r') as csvfile:
        pairs = []
        csvreader = csv.reader(csvfile, delimiter=',')
        for line in csvreader:
            try:
                pairs.append([normalize_string(line[1]), normalize_string(line[2])])
            except IndexError:
                pass
    
    # Reverse pairs, make Lang instances (not supported)
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        voc = Voc(csvname)
    else:
        voc = Voc(csvname)

    return voc, pairs

In [7]:
def read_stories(pathname, reverse=False, num_files=100):
    print("Reading csv...")
    
    # Get the first num_files file names
    files = os.listdir('../data/%s' % pathname)[:num_files]
    
    # Boolean flag for summary sentences
    summary_next = False
    
    # Read each file and extract vocab
    pairs = []
    for filename in files:
        with open('../data/%s/%s' % (pathname, filename), 'r') as story_file:
            story = ''
            summary = ''
            for line in story_file:
                if line == '\n': continue # ignore blank lines
                if line == '@highlight\n': summary_next = True
                else:
                    if summary_next: summary += line
                    else: story += line
                    summary_next = False
            pairs.append([normalize_string(story), normalize_string(summary)])
            
    # Reverse pairs, make Lang instances (not supported)
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
    voc = Voc(pathname)

    return voc, pairs

## Preparing Data
Following is the main pre-processing function. Reads some articles and generates article-summary pairs and a comprehensive vocabulary from the articles.

In [8]:
def prepare_data(pathname, reverse=False):
    voc, pairs = read_stories(pathname, reverse)
    print("Read %s article-summary pairs" % len(pairs))

    print("Indexing words...")
    for pair in pairs:
        voc.index_words(pair[0])
        voc.index_words(pair[1])
    print("    found %d words" % voc.n_words)

    return voc, pairs

voc, pairs = prepare_data('dm_stories_tokenized', False)

pair = random.choice(pairs)
print(pair[0]+'\n')
print(pair[1])

Reading csv...
Read 100 article-summary pairs
Indexing words...
    found 9110 words
as a young girl chelsea clinton learned to keep secrets . but she also learned to call the secret service pigs . that s a claim from former white house florist ronn payne retold in a new book based on interviews with more than members of the presidential mansion s domestic staff . as he walked into the second floor kitchen one day he saw chelsea talking on the phone . a member of her secret service protective detail came in behind him to take the clintons only child to school . oh i ve got to go . the pigs are here she told her phone pal according to payne using a s era epithet for law enforcement . faced with an angry agent who reminded her in no uncertain terms that it was his job to protect her chelsea replied well that s what my mother and father call you . scroll down for video pigs chelsea clinton lrb right in rrb allegedly referred to the secret service with an offensive counter culture epithet 

## Turning training data into Tensors/Variables

To train we need to turn the sentences into something the neural network can understand, which of course means numbers. Each sentence will be split into words and turned into a Tensor, where each word is replaced with the index (from the Lang indexes made earlier). While creating these tensors we will also append the EOS token to signal that the sentence is over.

![](https://i.imgur.com/LzocpGH.png)

A Tensor is a multi-dimensional array of numbers, defined with some type e.g. FloatTensor or LongTensor. In this case we'll be using LongTensor to represent an array of integer indexes.

Trainable PyTorch modules take Variables as input, rather than plain Tensors. A Variable is basically a Tensor that is able to keep track of the graph state, which is what makes autograd (automatic calculation of backwards gradients) possible.

In [9]:
# Return a list of indexes, one for each word in the sentence
def indexes_from_sentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')]

def variable_from_sentence(voc, sentence):
    indexes = indexes_from_sentence(voc, sentence)
    indexes.append(EOS_token)
    var = Variable(torch.LongTensor(indexes).view(-1, 1))
#     print('var =', var)
    if USE_CUDA: var = var.cuda()
    return var

def variables_from_pair(pair):
    input_variable = variable_from_sentence(voc, pair[0])
    target_variable = variable_from_sentence(voc, pair[1])
    return (input_variable, target_variable)

In [10]:
training_pair = variables_from_pair(random.choice(pairs))
input_variable = training_pair[0]
target_variable = training_pair[1]
print(input_variable[:5])

Variable containing:
 4326
  493
 1952
  634
   57
[torch.LongTensor of size 5x1]



# Building the models

## The Encoder

<img src="images/encoder-network.png" style="float: right" />

The encoder of a seq2seq network is a RNN that outputs some value for every word from the input sentence. For every input word the encoder outputs a vector and a hidden state, and uses the hidden state for the next input word.

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        
    def forward(self, word_inputs, hidden):
        # Note: we run this all at once (over the whole input sequence)
        seq_len = len(word_inputs)
        embedded = self.embedding(word_inputs).view(seq_len, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        if USE_CUDA: hidden = hidden.cuda()
        return hidden

## Attention Decoder

### Interpreting the Bahdanau et al. model

The attention model in [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473) is described as the following series of equations.

Each decoder output is conditioned on the previous outputs and some $\mathbf x$, where $\mathbf x$ consists of the current hidden state (which takes into account previous outputs) and the attention "context", which is calculated below. The function $g$ is a fully-connected layer with a nonlinear activation, which takes as input the values $y_{i-1}$, $s_i$, and $c_i$ concatenated.

$$
p(y_i \mid \{y_1,...,y_{i-1}\},\mathbf{x}) = g(y_{i-1}, s_i, c_i)
$$

The current hidden state $s_i$ is calculated by an RNN $f$ with the last hidden state $s_{i-1}$, last decoder output value $y_{i-1}$, and context vector $c_i$.

In the code, the RNN will be a `nn.GRU` layer, the hidden state $s_i$ will be called `hidden`, the output $y_i$ called `output`, and context $c_i$ called `context`.

$$
s_i = f(s_{i-1}, y_{i-1}, c_i)
$$

The context vector $c_i$ is a weighted sum of all encoder outputs, where each weight $a_{ij}$ is the amount of "attention" paid to the corresponding encoder output $h_j$.

$$
c_i = \sum_{j=1}^{T_x} a_{ij} h_j
$$

... where each weight $a_{ij}$ is a normalized (over all steps) attention "energy" $e_{ij}$ ...

$$
a_{ij} = \dfrac{exp(e_{ij})}{\sum_{k=1}^{T} exp(e_{ik})}
$$

... where each attention energy is calculated with some function $a$ (such as another linear layer) using the last hidden state $s_{i-1}$ and that particular encoder output $h_j$:

$$
e_{ij} = a(s_{i-1}, h_j)
$$

### Implementing the Bahdanau et al. model

In summary our decoder should consist of four main parts - an embedding layer turning an input word into a vector; a layer to calculate the attention energy per encoder output; a RNN layer; and an output layer.

The decoder's inputs are the last RNN hidden state $s_{i-1}$, last output $y_{i-1}$, and all encoder outputs $h_*$.

* embedding layer with inputs $y_{i-1}$
    * `embedded = embedding(last_rnn_output)`
* attention layer $a$ with inputs $(s_{i-1}, h_j)$ and outputs $e_{ij}$, normalized to create $a_{ij}$
    * `attn_energies[j] = attn_layer(last_hidden, encoder_outputs[j])`
    * `attn_weights = normalize(attn_energies)`
* context vector $c_i$ as an attention-weighted average of encoder outputs
    * `context = sum(attn_weights * encoder_outputs)`
* RNN layer(s) $f$ with inputs $(s_{i-1}, y_{i-1}, c_i)$ and internal hidden state, outputting $s_i$
    * `rnn_input = concat(embedded, context)`
    * `rnn_output, rnn_hidden = rnn(rnn_input, last_hidden)`
* an output layer $g$ with inputs $(y_{i-1}, s_i, c_i)$, outputting $y_i$
    * `output = out(embedded, rnn_output, context)`

In [12]:
class BahdanauAttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1):
        super(BahdanauAttnDecoderRNN, self).__init__()
        
        # Define parameters
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        # Define layers
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.attn = Attn('concat', hidden_size)
        self.gru = nn.GRU(hidden_size * 2, hidden_size, n_layers, dropout=dropout_p)
        self.out = nn.Linear(hidden_size * 2, output_size)
    
    def forward(self, word_input, last_hidden, encoder_outputs):
        # Note that we will only be running forward for a single decoder time step, but will use all encoder outputs
        
        # Get the embedding of the current input word (last output word)
        word_embedded = self.embedding(word_input).view(1, 1, -1) # S=1 x B x N
        word_embedded = self.dropout(word_embedded)
        
        # Calculate attention weights and apply to encoder outputs
        print(encoder_outputs.size())

        attn_weights = self.attn(last_hidden[-1], encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x 1 x N
        
        # Combine embedded input word and attended context, run through RNN
        rnn_input = torch.cat((word_embedded, context), 2)
        output, hidden = self.gru(rnn_input, last_hidden)
        
        # Final output layer
        output = output.squeeze(0) # B x N
        context = context.squeeze(0) # B x N
        
        print("Context size: ", context.size())
        print("RNN out size: ", output.size())
        print("Output cat Context size: ", torch.cat((output, context), 1).shape)
        print("Output size: ", self.out(torch.cat((output, context), 1)).shape)
        output = F.log_softmax(self.out(torch.cat((output, context), 1)))
        
        # Return final output, hidden state, and attention weights (for visualization)
        return output, hidden, attn_weights

### Interpreting the Luong et al. model(s)

[Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025) by Luong et al. describe a few more attention models that offer improvements and simplifications. They describe a few "global attention" models, the distinction between them being the way the attention scores are calculated.

The general form of the attention calculation relies on the target (decoder) side hidden state and corresponding source (encoder) side state, normalized over all states to get values summing to 1:

$$
a_t(s) = align(h_t, \bar h_s)  = \dfrac{exp(score(h_t, \bar h_s))}{\sum_{s'} exp(score(h_t, \bar h_{s'}))}
$$

The specific "score" function that compares two states is either *dot*, a simple dot product between the states; *general*, a a dot product between the decoder hidden state and a linear transform of the encoder state; or *concat*, a dot product between a new parameter $v_a$ and a linear transform of the states concatenated together.

$$
score(h_t, \bar h_s) =
\begin{cases}
h_t ^\top \bar h_s & dot \\
h_t ^\top \textbf{W}_a \bar h_s & general \\
v_a ^\top \textbf{W}_a [ h_t ; \bar h_s ] & concat
\end{cases}
$$

The modular definition of these scoring functions gives us an opportunity to build specific attention module that can switch between the different score methods. The input to this module is always the hidden state (of the decoder RNN) and set of encoder outputs.

In [16]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        
        self.method = method
        self.hidden_size = hidden_size
        
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)

        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.other = nn.Parameter(torch.FloatTensor(1, hidden_size))

    def forward(self, hidden, encoder_outputs):
        seq_len = len(encoder_outputs)

        # Create variable to store attention energies
        attn_energies = Variable(torch.zeros(seq_len)) # B x 1 x S
        if USE_CUDA: attn_energies = attn_energies.cuda()

        # Calculate energies for each encoder output
        for i in range(seq_len):
            attn_energies[i] = self.score(hidden, encoder_outputs[i])

        # Normalize energies to weights in range 0 to 1, resize to 1 x 1 x seq_len
        return F.softmax(attn_energies).unsqueeze(0).unsqueeze(0)
    
    def score(self, hidden, encoder_output):
        
        if self.method == 'dot':
            energy = torch.dot(hidden.view(-1), energy.view(-1))
            return energy
        
        elif self.method == 'general':
            energy = self.attn(encoder_output)
            energy = torch.dot(hidden.view(-1), energy.view(-1))
            return energy
        
        elif self.method == 'concat':
            energy = self.attn(torch.cat((hidden, encoder_output), 1))
            energy = torch.dot(self.other.view(-1), energy.view(-1))
            return energy

Now we can build a decoder that plugs this Attn module in after the RNN to calculate attention weights, and apply those weights to the encoder outputs to get a context vector.

### Testing the models

To make sure the Encoder and Decoder model are working (and working together) we'll do a quick test with fake word inputs:

In [17]:
encoder_test = EncoderRNN(10, 10, 2)
decoder_test = BahdanauAttnDecoderRNN(10, 10, 2)
print(encoder_test)
print(decoder_test)

encoder_hidden = encoder_test.init_hidden()
word_input = Variable(torch.LongTensor([1, 2, 3]))

# if torch.cuda.device_count() > 1:
#     encoder_test = nn.DataParallel(encoder_test)

if USE_CUDA:
    encoder_test.cuda()
    word_input = word_input.cuda()
encoder_outputs, encoder_hidden = encoder_test(word_input, encoder_hidden)

word_inputs = Variable(torch.LongTensor([1, 2, 3]))
decoder_attns = torch.zeros(1, 3, 3)
decoder_hidden = encoder_hidden
decoder_context = Variable(torch.zeros(1, decoder_test.hidden_size))

# if torch.cuda.device_count() > 1:
#     decoder_test = nn.DataParallel(decoder_test)

if USE_CUDA:
    decoder_test.cuda()
    word_inputs = word_inputs.cuda()
    decoder_context = decoder_context.cuda()
    

for i in range(3):
    #def forward(self, word_input, last_hidden, encoder_outputs):
    decoder_output, decoder_hidden, decoder_attn = decoder_test(word_inputs[i], 
                                                                                 decoder_hidden, 
                                                                                 encoder_outputs)
    print(decoder_output.size(), decoder_hidden.size(), decoder_attn.size())
    decoder_attns[0, i] = decoder_attn.squeeze(0).cpu().data
    
    

EncoderRNN(
  (embedding): Embedding(10, 10)
  (gru): GRU(10, 10, num_layers=2)
)
BahdanauAttnDecoderRNN(
  (embedding): Embedding(10, 10)
  (dropout): Dropout(p=0.1)
  (attn): Attn(
    (attn): Linear(in_features=20, out_features=10)
  )
  (gru): GRU(20, 10, num_layers=2, dropout=0.1)
  (out): Linear(in_features=20, out_features=10)
)
torch.Size([3, 1, 10])
Context size:  torch.Size([1, 10])
RNN out size:  torch.Size([1, 10])
Output cat Context size:  torch.Size([1, 20])
Output size:  torch.Size([1, 10])
torch.Size([1, 10]) torch.Size([2, 1, 10]) torch.Size([1, 1, 3])
torch.Size([3, 1, 10])


/home/kabirm2/anaconda3/envs/pytorchcuda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/kabirm2/anaconda3/envs/pytorchcuda/lib/python3.6/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Context size:  torch.Size([1, 10])
RNN out size:  torch.Size([1, 10])
Output cat Context size:  torch.Size([1, 20])
Output size:  torch.Size([1, 10])
torch.Size([1, 10]) torch.Size([2, 1, 10]) torch.Size([1, 1, 3])
torch.Size([3, 1, 10])
Context size:  torch.Size([1, 10])
RNN out size:  torch.Size([1, 10])
Output cat Context size:  torch.Size([1, 20])
Output size:  torch.Size([1, 10])
torch.Size([1, 10]) torch.Size([2, 1, 10]) torch.Size([1, 1, 3])


In [18]:
a = torch.zeros(1,10)
b = torch.zeros(1,10)
torch.cat((a,b), dim=1)



Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 19 
    0     0     0     0     0     0     0
[torch.FloatTensor of size 1x20]

# Training

## Defining a training iteration

To train we first run the input sentence through the encoder word by word, and keep track of every output and the latest hidden state. Next the decoder is given the last hidden state of the decoder as its first hidden state, and the `<SOS>` token as its first input. From there we iterate to predict a next token from the decoder.

### Teacher Forcing and Scheduled Sampling

"Teacher Forcing", or maximum likelihood sampling, means using the real target outputs as each next input when training. The alternative is using the decoder's own guess as the next input. Using teacher forcing may cause the network to converge faster, but [when the trained network is exploited, it may exhibit instability](http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf).

You can observe outputs of teacher-forced networks that read with coherent grammar but wander far from the correct translation - you could think of it as having learned how to listen to the teacher's instructions, without learning how to venture out on its own.

The solution to the teacher-forcing "problem" is known as [Scheduled Sampling](https://arxiv.org/abs/1506.03099), which simply alternates between using the target values and predicted values when training. We will randomly choose to use teacher forcing with an if statement while training - sometimes we'll feed use real target as the input (ignoring the decoder's output), sometimes we'll use the decoder's output.

In [19]:
print(voc.n_words)

9110


In [20]:
teacher_forcing_ratio = 0.5
clip = 5.0

def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):

    # Zero gradients of both optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0 # Added onto for each word

    # Get size of input and target sentences
    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]

    # Run words through encoder
    encoder_hidden = encoder.init_hidden()
    encoder_outputs, encoder_hidden = encoder(input_variable, encoder_hidden)
    
    # Prepare input and output variables
    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_context = Variable(torch.zeros(1, decoder.hidden_size))
    decoder_hidden = encoder_hidden # Use last hidden state from encoder to start decoder
    if USE_CUDA:
        decoder_input = decoder_input.cuda()
        decoder_context = decoder_context.cuda()

    # Choose whether to use teacher forcing
    use_teacher_forcing = random.random() < teacher_forcing_ratio
    if use_teacher_forcing:
        
        # Teacher forcing: Use the ground-truth target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, 
                                                                            decoder_hidden, encoder_outputs)
            print("Decoder output[0]: ", decoder_output[0])
            print("Target variable[0]: ", target_variable[0])
            ###########################################
            # print(decoder_output.size(), target_variable[di].size())
            
            print(decoder_output.shape, target_variable[di].shape)
            loss += criterion(decoder_output, target_variable[di])
            decoder_input = target_variable[di] # Next target is next input

    else:
        # Without teacher forcing: use network's own prediction as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            #print(criterion(decoder_output[0], 
            #                target_variable[di]).size())
            
            ###########################################
            # print(decoder_output.size(), target_variable[di].size())
            print("Decoder output[0]: ", decoder_output[0])
            print("Target variable[0]: ", target_variable[0])
            print(decoder_output.shape, target_variable[di].shape)
            loss += criterion(decoder_output, target_variable[di])
            
            # Get most likely word index (highest value) from output
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]
            
            decoder_input = Variable(torch.LongTensor([[ni]])) # Chosen word is next input
            if USE_CUDA: decoder_input = decoder_input.cuda()

            # Stop at end of sentence (not necessary when using known targets)
            if ni == EOS_token: break

    # Backpropagation
    loss.backward()
    torch.nn.utils.clip_grad_norm(encoder.parameters(), clip)
    torch.nn.utils.clip_grad_norm(decoder.parameters(), clip)
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0] / target_length

Finally helper functions to print time elapsed and estimated time remaining, given the current time and progress.

In [21]:
def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

## Running training

With everything in place we can actually initialize a network and start training.

To start, we initialize models, optimizers, and a loss function (criterion).

In [22]:
attn_model = 'general'
hidden_size = 500
n_layers = 2
dropout_p = 0.05

# Initialize models
encoder = EncoderRNN(voc.n_words, hidden_size, n_layers)
decoder = BahdanauAttnDecoderRNN(hidden_size, voc.n_words, n_layers, dropout_p=dropout_p)

# Move models to GPU
if USE_CUDA:
    encoder.cuda()
    decoder.cuda()

# Initialize optimizers and criterion
learning_rate = 0.0001
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

Then set up variables for plotting and tracking progress:

In [20]:
# Configuring training
n_epochs = 500
plot_every = 10
print_every = 20

# Keep track of time elapsed and running averages
start = time.time()
plot_losses = []
print_loss_total = 0 # Reset every print_every
plot_loss_total = 0 # Reset every plot_every

To actually train, we call the train function many times, printing a summary as we go.

*Note:* If you run this notebook you can train, interrupt the kernel, evaluate, and continue training later. You can comment out the lines above where the encoder and decoder are initialized (so they aren't reset) or simply run the notebook starting from the following cell.

In [21]:
# Begin!
for epoch in range(1, n_epochs + 1):
    
    # Get training data for this cycle
    training_pair = variables_from_pair(random.choice(pairs))
    input_variable = training_pair[0]
    target_variable = training_pair[1]

    # Run the train function
    loss = train(input_variable, target_variable, encoder, 
                 decoder, encoder_optimizer, 
                 decoder_optimizer, 
                 criterion)

    # Keep track of loss
    print_loss_total += loss
    plot_loss_total += loss

    if epoch == 0: continue

    if epoch % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print_summary = '%s (%d %d%%) %.4f' % (time_since(start, epoch / n_epochs), epoch, epoch / n_epochs * 100, print_loss_avg)
        print(print_summary)

    if epoch % plot_every == 0:
        plot_loss_avg = plot_loss_total / plot_every
        plot_losses.append(plot_loss_avg)
        plot_loss_total = 0

torch.Size([614, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 57
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([614, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 57
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([614, 1, 500])


/home/kabirm2/anaconda3/envs/pytorchcuda/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/kabirm2/anaconda3/envs/pytorchcuda/lib/python3.6/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 57
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([614, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 57
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([614, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
n

Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 57
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([614, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 57
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([614, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
n

torch.Size([902, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 57
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([902, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 57
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([902, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  

...

Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 5213
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([1217, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 5213
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([1217, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:

Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 5213
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([1217, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 5213
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([1217, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:

Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 5213
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([1217, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 5213
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([1217, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:

Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 5213
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([1217, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:
nan
nan
nan
 ⋮ 
nan
nan
nan
[torch.FloatTensor of size 9110]

Target variable[0]:  Variable containing:
 5213
[torch.LongTensor of size 1]

torch.Size([1, 9110]) torch.Size([1])
torch.Size([1217, 1, 500])
Context size:  torch.Size([1, 500])
RNN out size:  torch.Size([1, 500])
Output cat Context size:  torch.Size([1, 1000])
Output size:  torch.Size([1, 9110])
Decoder output[0]:  Variable containing:

KeyboardInterrupt: 

## Plotting training loss

Plotting is done with matplotlib, using the array `plot_losses` that was created while training.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline

def show_plot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2) # put ticks at regular intervals
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

show_plot(plot_losses)

# Evaluating the network

Evaluation is mostly the same as training, but there are no targets. Instead we always feed the decoder's predictions back to itself. Every time it predicts a word, we add it to the output string. If it predicts the EOS token we stop there. We also store the decoder's attention outputs for each step to display later.

In [ ]:
def evaluate(sentence, max_length=200):
    input_variable = variable_from_sentence(voc, sentence)
    input_length = input_variable.size()[0]
    
    # Run through encoder
    encoder_hidden = encoder.init_hidden()
    encoder_outputs, encoder_hidden = encoder(input_variable, encoder_hidden)

    # Create starting vectors for decoder
    decoder_input = Variable(torch.LongTensor([[SOS_token]])) # SOS
    decoder_context = Variable(torch.zeros(1, decoder.hidden_size))
    if USE_CUDA:
        decoder_input = decoder_input.cuda()
        decoder_context = decoder_context.cuda()

    decoder_hidden = encoder_hidden
    
    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)
    
    # Run through decoder
    for di in range(max_length):
        decoder_output, decoder_context, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_context, decoder_hidden, encoder_outputs)
        decoder_attentions[di,:decoder_attention.size(2)] += decoder_attention.squeeze(0).squeeze(0).cpu().data

        # Choose top word from output
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(voc.index2word[ni])
            
        # Next input is chosen word
        decoder_input = Variable(torch.LongTensor([[ni]]))
        if USE_CUDA: decoder_input = decoder_input.cuda()
    
    return decoded_words, decoder_attentions[:di+1, :len(encoder_outputs)]

We can evaluate random sentences from the training set and print out the input, target, and output to make some subjective quality judgements:

In [ ]:
def evaluate_randomly():
    pair = random.choice(pairs)
    
    output_words, decoder_attn = evaluate(pair[0])
    output_sentence = ' '.join(output_words)
    
    print('>', pair[0])
    print('=', pair[1])
    print('<', output_sentence)
    print('')

In [ ]:
evaluate_randomly()

# Visualizing attention

A useful property of the attention mechanism is its highly interpretable outputs. Because it is used to weight specific encoder outputs of the input sequence, we can imagine looking where the network is focused most at each time step.

You could simply run `plt.matshow(attentions)` to see attention output displayed as a matrix, with the columns being input steps and rows being output steps:

In [ ]:
output_words, attentions = evaluate("house speaker who orchestrated the republican revolution of recent years and is overseeing the impeachment inquiry into president clinton was driven from office friday by a party that swiftly turned on him after its unexpected losses in tuesday s mid term elections .")
plt.matshow(attentions.numpy())

For a better viewing experience we will do the extra work of adding axes and labels:

In [ ]:
def show_attention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    fig.set_figwidth(20)
    
    # fig.set_figheight(50)

    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone', aspect = 1)
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

def evaluate_and_show_attention(input_sentence):
    output_words, attentions = evaluate(input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    show_attention(input_sentence, output_words, attentions)

In [ ]:
evaluate_and_show_attention("a south korean lawmaker said friday communist north korea could be producing bombs and could have more secret underground nuclear facilities than already feared .")

In [ ]:
pair = random.choice(pairs)
evaluate_and_show_attention(' '.join(pairs[0]))
#evaluate_and_show_attention("egyptian president hosni mubarak met here sunday with syrian president hafez assad to try to defuse growing tension between syria and turkey .")

In [ ]:
evaluate_and_show_attention("police and soldiers on friday blocked off the street in front of a house where members of a terrorist gang are believed to have assembled the bomb that blew up the u .s . embassy killing people .")

In [ ]:
evaluate_and_show_attention("premier battled tuesday for any votes freed up from a split in a far left party but said he will resign if he loses a confidence vote expected later this week .")

# To do

* Try with a different dataset
    * cnn/dailymail
    * gigawords
    * standford
    * Human &rarr; Machine (e.g. IOT commands)
    * Chat &rarr; Response
    * Question &rarr; Answer
* Replace the embedding pre-trained word embeddings such as word2vec or GloVe
* Try with more layers, more hidden units, and more sentences. Compare the training time and results.
* Try different RNN layers like lstm.
* Add batch operation for GPU training
* Add beam search on decoder side when dealing with long documents.
* Control the Different output size
* Dig out other tricks